In [ ]:
# [x] see if we have multi query capabilities
# [x] expose connection radius infrastructure
    # what we want:
    # [x] we want to create new states first with a specific connection radius - implement a ConnectionFilter instead of trying to import BoundedConnectionStrategy
    # [x] then we want to reconfigure the planner to only create new states when adding start and goals without growing the roadmap more (just 'using') - clear input query every time
# [x] decide on experiment apparatus
    # Run from d: 2....20
    # Naive: Pre-constructed instance (empty, narrow corridor, etc.). Then run with m (~500>) different roadmaps. Each roadmap is a single Monte-Carlo sample.
    # Naive: Run n trials on one roadmap <-- problem: number of trials we need to run scales alongside epsilon nets. ironic! ~either sample new states or just reuse graph states? sample lots of states and reuse?
# [ ] write up experiment apparatus
    # [x] switch to manually defining problems instead of simple setup since we need more control
    # [x] define validation of ONE d-dimensional roadmap
    # [x] add in multiple d-dimensional roadmaps
    # [x] add in control of graph growth in solution criterion
    # [x] add in parameter/n prob solution infrastructure
        # AS SAMPLES INCREASE m inc.
            # maintain a list of tolerances
    # [x] add in pandas dataframe charting infrastructure
        # current array format: N_trial x M_sample x T_tol -> # frac paths rel opt
# [ ] design parameters and run experiment
    # --what is likely going to happen is that given the current level of control we have form the python bindings, since we can only query individual paths, it's going to be too slow.
    # --run anyway so we can present data to nick
    # --but for interpretability, _now_ formally prove the result that means `if we want uniform converage with \epsilon-optimality' => we need an \epsilon'-net

In [8]:
%env OMPL_PATH=/home/seiji/Research/ompl/py-bindings
from nonasymptotic import ompl
from ompl import base as ob
from ompl import geometric as og

import numpy as np
import pandas as pd
from datetime import datetime

env: OMPL_PATH=/home/seiji/Research/ompl/py-bindings


In [2]:
# defining free parameters: dimension, obstacles (i.e. state validity)
sample_schedule = [10, 100, 1000, 10000, 100000, 1000000]
tols = np.array([0.5, 0.25, 0.1, 0.05])
n_trials = 5000
d = 3
r = 0.5
n_paths = 1000 # should be a function of dimensionality in the future for coverage
is_state_valid = lambda state: True

In [6]:
# set up unit cube space with obstacles
space = ob.RealVectorStateSpace(d)
space.setBounds(0.0, 1.0)

si = ob.SpaceInformation(space)
si.setStateValidityChecker(ob.StateValidityCheckerFn(is_state_valid))
pdef = ob.ProblemDefinition(si)

# set up the PRM planner
planner = og.PRM(si)
accept_if_within_r = lambda v1, v2: planner.distanceFunction(v1, v2) <= r
planner.setConnectionFilter(og.ConnectionFilter(accept_if_within_r))
planner.setProblemDefinition(pdef)

rec = np.array((n_trials, len(sample_schedule), tols.size))

for i_trial in range(n_trials):
    planner.clear()
    pdef.clearStartStates()
    pdef.clearGoal()

    # save fraction of rel-optness
    for i_sample, m_samples in enumerate(sample_schedule):
        num_rel_opt_paths = np.zeros_like(tols)
        graph_has_m_samples = lambda: planner.milestoneCount() >= m_samples
        term_cond = ob.PlannerTerminationConditionFn(graph_has_m_samples)
        for i_path in range(n_paths):
            # set up problem instance
            start = ob.State(space)
            start.random()

            goal = ob.State(space)
            goal.random()

            # compute optimum
            opt_len = planner.distanceFunction(start, goal)
            planner.clearQuery() # clear previous queries so we can multi-query
            pdef.setStartAndGoalStates(start, goal)

            planner.setup() # no harm in recalling if setup already
            solved = planner.solve(term_cond)

            if solved:
                path = pdef.getSolutionPath()
                length = path.length()

                num_rel_opt_paths += np.less_equal(length, (1 + tols) * opt_len, dtype=np.int)

        rec[i_trial, i_sample, :] = num_rel_opt_paths

SyntaxError: unexpected EOF while parsing (1798147844.py, line 34)

In [ ]:
# save data
mc = pd.MultiIndex.from_product([sample_schedule, tols], names=['n samples', 'tolerance'])
d = pd.DataFrame(rec.reshape((n_trials, -1), order='C'), index=range(n_trials), columns=mc)
d.to_pickle('data/prm_%s_%d.pkl' % (datetime.now().strftime('%m-%d-%Y'), d))